In [1]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from didcomm.message import Attachment, AttachmentDataJson
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

## Helper function

In [2]:
# %load /Users/alex/Projects/rootid/didcomm-mediator/db_utils.py
''' DB Utilities '''
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import os
import urllib.parse


mongo = MongoClient(
    'mongodb://localhost:27017',
    username=urllib.parse.quote_plus(os.environ["MONGODB_USER"]) if "MONGODB_USER" in os.environ else None,
    password=urllib.parse.quote_plus(os.environ["MONGODB_PASSWORD"]) if "MONGODB_PASSWORD" in os.environ else None,
    authSource="admin"
)


db = mongo.mediator

def get_connection(remote_did):
    ''' Get existing connection '''
    return db.connections.find_one({"remote_did": remote_did})

def create_connection(remote_did, local_did):
    ''' Create connection'''
    db.connections.insert_one({
        "remote_did": remote_did,
        "local_did": local_did,
        "creation_time": int(datetime.datetime.now().timestamp())
    })

def update_connection(remote_old_did, remote_new_did):
    ''' Update connection '''
    #TODO history of rotations
    db.connections.update_one(
        {"remote_did": remote_old_did}, {
            "$set": {"remote_did": remote_new_did, "update_time": int(datetime.datetime.now().timestamp())}
        }
    )

def get_oob_did():
    return db.oobs.find_one(sort=[('date', -1)])

def store_oob_did(did):
    db.oobs.insert_one(did)

def get_issuer_did():
    issuer_did = db.issuers.find_one(sort=[('date', -1)])
    return issuer_did if issuer_did is not None else None

def store_issuer_did(did):
    db.issuers.insert_one(did)


def store_vc(vc):
    db.vcs.insert_one(vc)

def get_vc(vc_id):
    return db.vcs.find_one({"id": vc_id})

def get_prism_holder_did():
    holder_did = db.prism_holder.find_one(sort=[('date', -1)])
    return holder_did if holder_did is not None else None

def store_prism_holder_did(did):
    db.prism_holder.insert_one(did)

def get_prism_did(did):
    return db.prism.find_one({"did": did})

def store_prism_did(did,seed):
    db.prism.insert_one({
        "did": did,
        "seed": seed
    })

def add_mediation(remote_did, routing_did, endpoint):
    ''' Add mediation info to connection '''
    db.connections.update_one(
        {"remote_did": remote_did}, {
            "$set": {
                        "isMediation": True, 
                        "routing_did": routing_did,
                        "endpoint": endpoint
                    }
        }
    )

def update_keys(remote_did, updates):
    ''' Add mediation keys '''
    connection = get_connection(remote_did)
    current_keys = connection["keylist"] if "keylist" in connection else []
    updated = []
    for update in updates:
        if update["action"] == "add":
            if update["recipient_did"] in current_keys:
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "add",
                        "result": "no_change"
                    })
            else:
                current_keys.append(update["recipient_did"])
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "add",
                        "result": "success"
                    })
        elif update["action"] == "remove":
            if update["recipient_did"] in current_keys:
                current_keys.remove(update["recipient_did"])
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "remove",
                        "result": "success"
                    })
            else:
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "remove",
                        "result": "no_change"
                    })
        else:
            updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": update["action"],
                        "result": "client_error"
                    })
        db.connections.update_one(
        {"remote_did": remote_did}, {
            "$set": {
                        "keylist": current_keys, 
                    }
        }
    )
    return updated

def get_message_status(remote_did, recipient_key):
    if recipient_key:
        count = db.messages.count_documents({"recipient_key": recipient_key},{"attachment":1})
    else:
        recipient_keys = db.connections.find_one({"remote_did":remote_did})["keylist"]
        count = db.messages.count_documents({"recipient_key": {"$in":recipient_keys}})
    return count

def get_messages(remote_did, recipient_key,limit):
    if recipient_key:
        return db.messages.find({"recipient_key": recipient_key},{"attachment":1}).limit(limit)
    else:
        recipient_keys = db.connections.find_one({"remote_did":remote_did})["keylist"]
        return list(db.messages.find({"recipient_key": {"$in":recipient_keys}}).limit(limit))

def add_message(recipient_key, attachment):
    # TODO verify that recipient_key belong to a registered peer
    db.messages.insert_one(
            {
                "recipient_key": recipient_key,
                "attachment": attachment,
                "datetime": int(datetime.datetime.now().timestamp())
            }
        )

def remove_messages(remote_did, message_id_list):
    #TODO verify that recipient_key belongs to remote_did
    for id in message_id_list:
        db.messages.delete_one({"_id": ObjectId(id)})
    return get_message_status(remote_did, None)

def get_short_url(oobid):
    short_url = db.shortUrls.find_one({"oobid": oobid},sort=[('date', -1)])
    return short_url

def store_short_url(short_url):
    db.shortUrls.insert_one(short_url)

def expire_short_url(oobid):
        # TODO order by date
        db.shortUrls.update_one(
            {"oobid": oobid}, {
                "$set": {
                            "expires_time": int(0)
                        }
            }
        )
    

In [3]:
secrets_resolver = SecretsResolverDemo()

In [4]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [5]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Reading issuer OOB message

In [6]:
# oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
print(oob_url)

http://127.0.0.1:8000?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiOWYyMDc5ZTItNmEzNS00OWRiLWE3M2MtY2MyZjAyNDc0YTU0IiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNnU2kxVm1rS01UY1dLSk1meHpKZ1ExUVFZbVZWVVhXeHlqQVN1c0Q2dTZQRC5WejZNa2d4dG5BRWpmc2FYcnl1SFNWWHRvN2t3V3NuRWs1TnAyU1hkVVdvZ1A3OWRBLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwiYWNjZXB0IjpbImRpZGNvbW0vdjIiLCJkaWRjb21tL2FpcDI7ZW52PXJmYzU4NyJdfX0


In [7]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '9f2079e2-6a35-49db-a73c-cc2f02474a54', 'from': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'request-mediate', 'goal': 'RequestMediate', 'accept': ['didcomm/v2', 'didcomm/aip2;env=rfc587']}}


## Prepare request-credential

The holder needs to create a did:peer to communicate with the issuer:

In [8]:
holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LSk8bHZFndDZAeuJwmCgdg6BSU9PtTxKSAAFdwbp9F8pgs.Vz6Mkj7QGyJkvgrqo8fNc5EEoKJody631ifGFGmAHXQRaQDAd.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


Also, the holder needs to provide a did:prism where the credential will be issued to.

In [9]:
holder_prism_did = get_prism_holder_did()['did']


The following is a JSON-LD credential request that the holder will submmit to the issuer.

In [10]:
credential_request = {
    "credential": {
        "@context": 
        [
            "https://www.w3.org/2018/credentials/v1",
            "https://www.w3.org/2018/credentials/examples/v1"
        ],
        "id": 'local-id',
        "type": ["VerifiableCredential", "UniversityDegreeCredential"],
        "issuer": "did:prism:fb727d449bd219021503d3f8c63a221cc1861a90e93b5c017b139aafdd2c14d9:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQNJuG3G_ZCEg9YHAXQpvLagXQTb8emtVTjBh_m66F5uoRI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhAiRByhWe4k1MTd7cbmRxrthrV7WPsRaKccZeDIAWoGJlEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiEDTkb_zwgk-ILQcH3tW3kCAiuuGcRcjiDlQ3h8eS0MmPk",
        "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "credentialSubject": 
            {
              "id": holder_prism_did,
              "degree": 
                {
                    "type": "BachelorDegree",
                    "name": "Bachelor of Science and Arts"
                  }
            },
        "options": {
            "proofType": "EcdsaSecp256k1Signature2019"
        }
    }
}

Finally the holder prepares the didcomm message following issue-credential/3.0 protocol:

In [11]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/request-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {
        "goal_code": "issue-credential",
        "comment": "some comment"
    },
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/json",
                format= "aries/ld-proof-vc-detail@v1.0",
                data=AttachmentDataJson(json=credential_request)
                )
    ]
                        
)

In [12]:
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

From the issuer DID obtained in the OOB QR Code, we can get the DID document:

In [13]:
issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
print(issuer_did_doc)

{'id': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'authentication': [{'id': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'publicKeyMultibase': 'z6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsI

And from there get the issuer's endpoint

In [14]:
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)

http://127.0.0.1:8000


# Finally sending the request to the issuer:
## takes a couple of minutes

In [16]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, data = holder_packed_msg.packed_msg)
resp

<Response [202]>

### Issued Verifiable Credential Received

### Issued Verifiable Credential Received

In [17]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

In [18]:
print(credential_unpack_msg)

UnpackResult(message=Message(id='83fde0c8-331a-4f77-ac7c-2e523080e957', type='https://didcomm.org/issue-credential/3.0/issue-credential', body={'goal_code': 'issue-credential', 'comment': 'some comment'}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid=None, pthid=None, attachments=[Attachment(data=AttachmentDataJson(json={'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], 'id': '3817d040-5d8d-4a3c-8241-66679f55f2e0', 'type': ['VerifiableCredential', 'UniversityDegreeCredential'], 'issuer': 'did:prism:fb727d449bd219021503d3f8c63a221cc1861a90e93b5c017b139aafdd2c14d9:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQNJuG3G_ZCEg9YHAXQpvLagXQTb8emtVTjBh_m66F5uoRI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhAiRByhWe4k1MTd7cbmRxrthrV7WPsRaKccZeDIAWoGJlEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiEDTkb_zwgk-ILQcH3tW3kCAiuuGcRcjiDlQ3h8eS0MmPk', 'issuanceDate': '2022-09-19T12:36:58Z', 'credentialS

In [19]:
credential = credential_unpack_msg.message.attachments[0].data.json

In [20]:
credential

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': '3817d040-5d8d-4a3c-8241-66679f55f2e0',
 'type': ['VerifiableCredential', 'UniversityDegreeCredential'],
 'issuer': 'did:prism:fb727d449bd219021503d3f8c63a221cc1861a90e93b5c017b139aafdd2c14d9:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQNJuG3G_ZCEg9YHAXQpvLagXQTb8emtVTjBh_m66F5uoRI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhAiRByhWe4k1MTd7cbmRxrthrV7WPsRaKccZeDIAWoGJlEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiEDTkb_zwgk-ILQcH3tW3kCAiuuGcRcjiDlQ3h8eS0MmPk',
 'issuanceDate': '2022-09-19T12:36:58Z',
 'credentialSubject': {'id': 'did:prism:8ade34754487af867b813d008d3704451a0c1df88090c1947d6f901857fd5ada:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQJ-gKvROnRTlmvLtBGxikIg07oaJZc2B5bU4MrAGtwDkhI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhA6I4nH3zH1vn1Azo_WLGr4Btz5eb9cHA-KdQq5lIsGUoEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiECJiZwIzuQQen4LL-OPOBUCs9zsrD0u2Z_q2PhZzx0Nrc',
  'degree': {'type': 

# Request VP

In [21]:
oob_url = requests.get("http://127.0.0.1:8000/verify_url").text
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '599f3638-b563-4937-9487-dfe55099d900', 'from': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'streamlined-vp', 'accept': ['didcomm/v2']}}


# PROPOSE PRESENTATION

In [22]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/present-proof/3.0/propose-presentation",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {}
)
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
verifier_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
verifier_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application     /didcomm-encrypted+json"}
resp = requests.post(verifier_endpoint, headers=headers, data = holder_packed_msg.packed_msg)
resp


<Response [202]>

In [23]:
presentation_definition_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
presentation_definition =presentation_definition_msg.message.attachments[0].data.json
presentation_definition

{'presentation_definition': {'presentation_definition': {'id': '32f54163-7166-48f1-93d8-ff217bdb0654',
   'input_descriptors': [{'id': '66685f64-7166-5717-b3fc-ff217bdb9999',
     'constraints': {'fields': [{'path': ['$.type'],
        'filter': {'type': 'string',
         'pattern': 'UniversityDegreeCredential'}}]}}]}},
 'options': {'challenge': '3fa85f64-5717-4562-b3fc-2c963f66afa7',
  'domain': '4jt78h47fh47'}}

# Create a Presentation Submission
from `presentation_definition` and `credential`

In [26]:
import hashlib, ecdsa
from pyld import jsonld

def sign(credential, presentation_definition):
    hex_private_holder = get_prism_holder_did()['master0']
    sk = ecdsa.SigningKey.from_string(bytes.fromhex(hex_private_holder), curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256) # the default is sha1

    proof_hash = hashlib.sha256(presentation_definition['options']['challenge'].encode('utf-8'))
    normalized_proof = jsonld.normalize(credential, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
    proof_hash.update(normalized_proof.encode('utf-8'))

    sig = sk.sign(bytes.fromhex(proof_hash.hexdigest()))
    return sig.hex()
sign(credential, presentation_definition)

'2268a85c0d2292a827d8e55c09f03ceec7680a5110a1d5bae42089bc9e325603a3a1132db40f2fbdda9b0ab3a278db0d4a94a77a13cf5c6d75a8a604305a0c84'

In [27]:
submission = {
          "@context": [
            "https://www.w3.org/2018/credentials/v1",
            "https://identity.foundation/presentation-exchange/submission/v1"
          ],
          "type": [
            "VerifiablePresentation",
            "PresentationSubmission"
          ],
          "holder": "did:example:123",
          "verifiableCredential": [
            credential
            ],
          "presentation_submission":{"definition_id":presentation_definition['presentation_definition']['presentation_definition']['id'] },
          "proof":{
            "type": "EcdsaSecp256k1Signature2019",
            "created": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
            "verificationMethod": get_prism_holder_did()['did'] + "#master0",
            "proofPurpose": "assertionMethod",
            "challenge": presentation_definition['options']['challenge'],
            "proofValue": sign(credential, presentation_definition)
          }
}
submission

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://identity.foundation/presentation-exchange/submission/v1'],
 'type': ['VerifiablePresentation', 'PresentationSubmission'],
 'holder': 'did:example:123',
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v1',
    'https://www.w3.org/2018/credentials/examples/v1'],
   'id': '3817d040-5d8d-4a3c-8241-66679f55f2e0',
   'type': ['VerifiableCredential', 'UniversityDegreeCredential'],
   'issuer': 'did:prism:fb727d449bd219021503d3f8c63a221cc1861a90e93b5c017b139aafdd2c14d9:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQNJuG3G_ZCEg9YHAXQpvLagXQTb8emtVTjBh_m66F5uoRI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhAiRByhWe4k1MTd7cbmRxrthrV7WPsRaKccZeDIAWoGJlEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiEDTkb_zwgk-ILQcH3tW3kCAiuuGcRcjiDlQ3h8eS0MmPk',
   'issuanceDate': '2022-09-19T12:36:58Z',
   'credentialSubject': {'id': 'did:prism:8ade34754487af867b813d008d3704451a0c1df88090c1947d6f901857fd5ada:Cr8BCrwBEjsKB21hc3RlcjAQAU

In [28]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/present-proof/3.0/presentation",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {},
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/ld+json",
                format= "dif/presentation-exchange/submission@v1.0",
                data=AttachmentDataJson(json=submission)
                )
    ]
)
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
verifier_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
verifier_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(verifier_endpoint, headers=headers, data = holder_packed_msg.packed_msg)
resp


<Response [202]>

In [29]:
presentation_definition_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

In [30]:
presentation_definition_msg.message.body

{'status': 'OK', 'proof-status': True, 'prism_credential_status': True}

In [31]:
presentation_definition_msg.message.type

'https://didcomm.org/present-proof/3.0/ack'